# Word Sense Disambiguation

In [ ]:
import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
import random
from collections import defaultdict

In [11]:
# Per assicurarsi di avere i corpora necessari
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('semcor')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\albia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\albia\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package semcor to
[nltk_data]     C:\Users\albia\AppData\Roaming\nltk_data...
[nltk_data]   Package semcor is already up-to-date!


True

Estrazione stopwords

In [31]:
stop_words = []
file_name = 'stop_words_FULL.txt'

with open(file_name, 'r', encoding='utf-8') as train:
   righe = train.readlines()
   for riga in righe:
      riga = riga.strip()
      stop_words.append(riga)
print(stop_words)

['a', 'able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'adopted', 'affected', 'affecting', 'affects', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently', 'approximately', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'available', 'away', 'awfully', 'b', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'brief', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can', 'cannot', "can't", 'cause', 'causes', 'certain', 'certainly', 'co', 'com', 'come', 'co

### Semcor
Estrazione di 50 frasi casuali 

In [46]:
def extract_sentences_from_semcor(n):
    sentences = []
    target_words = []
    annotated_senses = []

    for i in range(n):
        sentence = semcor.tagged_sents(tag='both')[i]
        words = []
        word = None
        sense = None

        for w in sentence:
            if w[0].lower() not in stop_words:        
                if isinstance(w, nltk.Tree) and w.label().split('.')[1] == 'n':
                    word = w.leaves()[0]
                    sense = w.label()
                    if sense is not None:
                        sense = wn.synset(sense)
                    break
                else:
                    words.append(w[0])
        
        if word and sense:
            sentences.append(words)
            target_words.append(word)
            annotated_senses.append(sense)
    
    return sentences, target_words, annotated_senses

### Lesk
Implementazione dell'algoritmo

In [13]:
def lesk_algorithm(context_sentence, ambiguous_word):
    max_overlap = 0
    best_sense = None
    context = set(nltk.word_tokenize(context_sentence))
    
    for sense in wn.synsets(ambiguous_word, pos=wn.NOUN):
        signature = set(nltk.word_tokenize(sense.definition()))
        for example in sense.examples():
            signature.update(nltk.word_tokenize(example))
        
        overlap = len(context.intersection(signature))
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    
    return best_sense

Valutazione dell'algoritmo

In [14]:
def evaluate_lesk(sentences, target_words, annotated_senses):
    correct = 0
    for i in range(len(sentences)):
        context_sentence = ' '.join(word for word in sentences[i])
        ambiguous_word = target_words[i]
        predicted_sense = lesk_algorithm(context_sentence, ambiguous_word)
        actual_sense = annotated_senses[i]
        
        if predicted_sense == actual_sense:
            correct += 1
    return correct / len(sentences)


Applicazione dell'algoritmo a frasi estratte casualmente

In [15]:
def randomize_evaluation(n, iterations):
    accuracies = []
    for _ in range(iterations):
        sentences, target_words, annotated_senses = extract_sentences_from_semcor(n)
        accuracy = evaluate_lesk(sentences, target_words, annotated_senses)
        accuracies.append(accuracy)
    
    return sum(accuracies) / len(accuracies)

In [48]:
# Eseguire l'algoritmo
num_sentences = 50
iterations = 10
average_accuracy = randomize_evaluation(num_sentences, iterations)
print(f"Average Accuracy over {iterations} iterations: {average_accuracy * 100:.2f}%")